Voy a hacer un test para disminuir al 10% el test y validacion para checar accuracy antes de correr todo el programa. 
- Cambio el batch a 8 en vez de 64 - eso le funcionaba a bien aqui https://www.kaggle.com/competitions/rsna-miccai-brain-tumor-radiogenomic-classification/discussion/281347
- Cambio el size a (224,224) otra vez pq tambien funcionaba para la competencia anterior

In [2]:
import pandas as pd
import numpy as np
import json 

import os
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom import dcmread
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import AUC, Precision, Recall

import matplotlib.pyplot as plt
import seaborn as sns 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

In [3]:
# Loading the dfs
train = pd.read_csv('../data/raw/train.csv')
train_label_coordinates = pd.read_csv('../data/raw/train_label_coordinates.csv')
train_series_descriptions = pd.read_csv('../data/raw/train_series_descriptions.csv')

train.head(3)

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,left_neural_foraminal_narrowing_l5_s1,right_neural_foraminal_narrowing_l1_l2,right_neural_foraminal_narrowing_l2_l3,right_neural_foraminal_narrowing_l3_l4,right_neural_foraminal_narrowing_l4_l5,right_neural_foraminal_narrowing_l5_s1,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [4]:
train_series_descriptions.head(3)

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2


In [5]:
spinal = train[['study_id', 'spinal_canal_stenosis_l1_l2', 'spinal_canal_stenosis_l2_l3', 
                'spinal_canal_stenosis_l3_l4', 'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1']]

# Changing the format to long 
spinal = spinal.melt(id_vars=['study_id'], var_name='level', value_name='severity')
spinal = spinal.sort_values(['study_id', 'level']).reset_index(drop=True)
spinal = spinal.dropna()
spinal.head()

,study_id,level,severity
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild
1,4003253,spinal_canal_stenosis_l2_l3,Normal/Mild
2,4003253,spinal_canal_stenosis_l3_l4,Normal/Mild
3,4003253,spinal_canal_stenosis_l4_l5,Normal/Mild
4,4003253,spinal_canal_stenosis_l5_s1,Normal/Mild


In [6]:
train_label_coordinates.head(3)

,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182


In [7]:
train_label = train_label_coordinates[train_label_coordinates['condition']=='Spinal Canal Stenosis']
spinal_merged = pd.merge(spinal, train_label, on='study_id', how='inner')
spinal_merged = pd.merge(spinal_merged, train_series_descriptions, on=['study_id', 'series_id'], how='inner')
spinal_merged.head()

,study_id,level_x,severity,series_id,instance_number,condition,level_y,x,y,series_description
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR


In [8]:
severity_unique = spinal_merged['severity'].unique()
severity_enc = dict(zip(severity_unique, range(len(severity_unique))))

# Saving the dictionary as a json file 
json.dump(severity_enc, open(f'../data/interim/enc_severity.json', 'w'))

spinal_merged['severity_enc'] = spinal_merged['severity'].map(severity_enc)
spinal_merged.head()
len(spinal_merged)

48765

In [9]:
# Getting a list of the studies needed for our model
studies = spinal_merged['study_id'].unique()
studies = [str(study) for study in studies]

main_path = "G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images"

dcm_paths = []

for study_folder in os.listdir(main_path):
    if study_folder in studies:
        study_path = os.altsep.join([main_path, study_folder])
        if os.path.isdir(study_path):
            for series_folder in os.listdir(study_path):
                series_path = os.altsep.join([study_path, series_folder])
                if os.path.isdir(series_path):
                    for dcm_file in os.listdir(series_path):
                        if dcm_file.endswith('.dcm'):      
                            dcm_paths.append({
                                'study_id': study_folder,
                                'series_id': series_folder,
                                'dcm_path': os.altsep.join([series_path, dcm_file])
                            })

dicom_df = pd.DataFrame(dcm_paths)
dicom_df = dicom_df.sort_values('study_id').reset_index(drop=True)
dicom_df['study_id'] = dicom_df['study_id'].astype('int64')
dicom_df['series_id'] = dicom_df['series_id'].astype('int64')
dicom_df.head(10)
len(dicom_df)

147131

In [10]:
spinal_merged = pd.merge(spinal_merged, dicom_df, on=['study_id', 'series_id'], how='inner')
spinal_merged.head()

,study_id,level_x,severity,series_id,instance_number,condition,level_y,x,y,series_description,severity_enc,dcm_path
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/12.dcm
1,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/9.dcm
2,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/8.dcm
3,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/7.dcm
4,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/6.dcm


In [11]:
len(spinal_merged)

829785

### Sample df for test

In [13]:
df_sample = spinal_merged.groupby('severity_enc', group_keys=False).apply(lambda x: x.sample(frac=0.05, random_state=123))
df_sample = df_sample.sample(frac=1, random_state=123)

C:\Users\noehu\AppData\Local\Temp\ipykernel_23960\1905865970.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = spinal_merged.groupby('severity_enc', group_keys=False).apply(lambda x: x.sample(frac=0.05, random_state=123))


In [14]:
df_sample.head()

,study_id,level_x,severity,series_id,instance_number,condition,level_y,x,y,series_description,severity_enc,dcm_path
743768,3859825215,spinal_canal_stenosis_l1_l2,Normal/Mild,4060210583,10,Spinal Canal Stenosis,L5/S1,225.152674,351.779515,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/3859825215/4060210583/14.dcm
598727,3110826149,spinal_canal_stenosis_l4_l5,Normal/Mild,1740265703,9,Spinal Canal Stenosis,L4/L5,327.082927,339.252033,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/3110826149/1740265703/6.dcm
219987,1140449293,spinal_canal_stenosis_l4_l5,Severe,1403088331,9,Spinal Canal Stenosis,L2/L3,250.571819,173.351573,Sagittal T2/STIR,2,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/1140449293/1403088331/8.dcm
390818,2040217841,spinal_canal_stenosis_l5_s1,Normal/Mild,3913032764,9,Spinal Canal Stenosis,L5/S1,216.372549,254.745098,Sagittal T2/STIR,0,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/2040217841/3913032764/3.dcm
710455,3685379620,spinal_canal_stenosis_l5_s1,Moderate,2214293322,8,Spinal Canal Stenosis,L4/L5,439.369176,435.125434,Sagittal T2/STIR,1,G:/RSNA/rsna-2024-lumbar-spine-degenerative-classification/train_images/3685379620/2214293322/10.dcm


In [15]:
# Original wgt
classes = np.unique(spinal_merged['severity_enc'])
class_wgt = compute_class_weight(class_weight='balanced', classes=classes, y=spinal_merged['severity_enc'])
class_wgt_dict = {c:w for c, w in zip(classes, class_wgt)}

class_wgt_dict

{0: 0.3817135743996114, 1: 4.330864622803996, 2: 6.696404793608522}

In [16]:
# Sample wgt
classes_samp = np.unique(df_sample['severity_enc'])
class_wgt_samp = compute_class_weight(class_weight='balanced', classes=classes_samp, y=df_sample['severity_enc'])
class_wgt_dict_samp = {c:w for c, w in zip(classes_samp, class_wgt_samp)}

class_wgt_dict_samp

{0: 0.38170811367797375, 1: 4.331245432717402, 2: 6.697175141242938}

In [17]:
# 1. Función para procesar archivos DICOM
def process_dicom(dcm_path, image_size=(224, 224)):
    dcm_path_str = dcm_path.numpy().decode("utf-8")
    dicom = pydicom.dcmread(dcm_path_str)
    pixel_array = dicom.pixel_array
    # Normalizar y redimensionar la imagen
    pixel_array = pixel_array / np.max(pixel_array)
    resized_image = tf.image.resize(pixel_array[..., np.newaxis], image_size)
    return tf.cast(resized_image, tf.float32)

# 2. Función para cargar y procesar imágenes
def load_and_preprocess(path, label):
    image = tf.py_function(func=process_dicom, inp=[path], Tout=tf.float32)
    image.set_shape((224, 224, 1))  # Fijo el shape pq me da error en el entrenamiento (wrong shape (224,224))
    return image, label

In [18]:
# 3. Crear tensor dataset
dataset = tf.data.Dataset.from_tensor_slices((df_sample["dcm_path"].values, df_sample["severity_enc"].values))
dataset = dataset.map(load_and_preprocess)
dataset_size = len(dataset)

In [19]:
# 5. Dividir dataset en entrenamiento y validación
train_size = int(0.8 * dataset_size)  # 80% para entrenamiento
val_size = dataset_size - train_size  # 20% para validación


train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Aplicar batching y prefetch después de dividir
train_dataset = train_dataset.batch(8).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

In [20]:
# 6. Verificar datasets
print(f"Tamaño del dataset de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del dataset de validación: {len(val_dataset)}")

Tamaño del dataset de entrenamiento: 4149
Tamaño del dataset de validación: 1038


In [21]:
n_classes = 3
train_dataset = train_dataset.map(lambda x, y: (x, to_categorical(y, num_classes=n_classes)))
val_dataset = val_dataset.map(lambda x, y: (x, to_categorical(y, num_classes=n_classes)))

In [22]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Lambda, GlobalAveragePooling2D, Dense
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Convertir imágenes de escala de grises (1 canal) a RGB (3 canales)
def preprocess_input_grayscale_to_rgb(input):
    return tf.image.grayscale_to_rgb(input)

# Construir el modelo base (ResNet50 preentrenado)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base para usar como extractor de características
base_model.trainable = False

# Crear el modelo final combinando el modelo preentrenado y capas personalizadas
model = models.Sequential([
    layers.Input(shape=(224, 224, 1)),              # Entrada de imágenes de 112x112x1
    Lambda(preprocess_input_grayscale_to_rgb),      # Convertir a RGB (112x112x3)
    base_model,                                    # Modelo preentrenado (ResNet50)
    GlobalAveragePooling2D(),                      # Reducción de dimensionalidad
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),          # Capa densa personalizada
    layers.Dense(3, activation='softmax')          # Tres clases de salida
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-5), 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(), 
                       tf.keras.metrics.Recall(), 
                       tf.keras.metrics.AUC()])

In [23]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5, class_weight=class_wgt_dict)

Epoch 1/5
4149/4149 ━━━━━━━━━━━━━━━━━━━━ 1318s 316ms/step - accuracy: 0.3809 - auc: 0.5655 - loss: 2.6642 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5359 - val_auc: 0.7783 - val_loss: 1.3248 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
4149/4149 ━━━━━━━━━━━━━━━━━━━━ 1300s 313ms/step - accuracy: 0.2954 - auc: 0.4827 - loss: 1.3281 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5564 - val_auc: 0.7948 - val_loss: 1.1718 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
4149/4149 ━━━━━━━━━━━━━━━━━━━━ 1392s 335ms/step - accuracy: 0.3240 - auc: 0.4978 - loss: 1.2445 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5779 - val_auc: 0.8095 - val_loss: 1.1454 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/5
4149/4149 ━━━━━━━━━━━━━━━━━━━━ 1308s 315ms/step - accuracy: 0.3485 - auc: 0.5186 - loss: 1.2241 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6068 - val_auc: 0.8261 - val_loss: 1.12